# Visualize and compare two GVEC states, using pygvec

In [ ]:
from pathlib import Path
import os
import numpy as np
import matplotlib.pyplot as plt

import gvec

In [ ]:
cwd = Path(os.getcwd())
print(f"current working directory:{cwd}")
cases = {}

# rundir=Path("../../../gvec/test-CI/run/example/")
# examples=[d.name for d in rundir.glob("*") if d.is_dir()]

examples = [
    "ellipstell_lowres",
    "ellipstell_lowres_restart_to_highres",
    "w7x_from_vmec_initLA_T",
    "w7x_from_nemec_highres",
]

# define "_a_" vs "_b_" cases
for example in examples:
    cases["_a_" + example] = {
        "rundir": "../../../gvec/test-CI/run_ref/example/" + example,
        "param": None,  # if None, "parameter.ini" is used
        "state": None,  # if None, last *State*.dat is used
        "title": "develop",
    }
    cases["_b_" + example] = {
        "rundir": "../../../gvec/test-CI/run/example/" + example,
        "param": None,
        "state": None,
        "title": "merge_frenet_pygvec",
    }

for case in cases.values():
    rundir = Path(case["rundir"])
    assert rundir.exists(), f"rundir:{rundir} does not exist"
    if case["param"] is None:
        case["param"] = rundir / "parameter.ini"
        assert (case["param"]).exists()
    if case["state"] is None:
        case["state"] = sorted(rundir.glob("*State*.dat"))[-1]

In [ ]:
# select an example

# example="ellipstell_lowres"
# example="ellipstell_lowres_restart_to_highres"
# example="w7x_from_vmec_initLA_T"
example = "w7x_from_nemec_highres"

case_a = cases["_a_" + example]
case_b = cases["_b_" + example]

print("comparison of states:")
print("-" * 50)
print(f" a: {case_a['state']}\n b: {case_b['state']}")

In [ ]:
# for visualization
nrho = 10
nthet = 32
nzeta = 22

for case in [case_a, case_b]:
    with gvec.State(case["param"], case["state"]) as state:
        # use integration grid
        ev_prof = gvec.Evaluations(rho="int", theta="int", zeta="int", state=state)
        state.compute(
            ev_prof,
            "iota",
            "p",
            "I_tor",
            "I_pol",
            "Phi",
            "chi",
            "F_r_avg",
            "N_FP",
            "V",
            "W_MHD",
        )
        # use visualization grid
        ev_visu = gvec.Evaluations(
            rho=np.linspace(1e-4, 1.0, nrho),
            theta=np.linspace(0.0, 2 * np.pi, nthet),
            zeta=np.linspace(0, 2 * np.pi, nzeta),
            state=state,
        )
        state.compute(ev_visu, "X1", "X2", "LA", "N_FP", "mod_B")

    case["ev_prof"] = ev_prof
    case["ev_visu"] = ev_visu

Va = case_a["ev_prof"].V.item()
Vb = case_b["ev_prof"].V.item()

print(
    f"compare volumes: |1-V(a)/V(b)|= {np.abs(1-Va/Vb):.4e}, V(a)={Va:.7e},V(b)={Vb:.7e},"
)
Wa = case_a["ev_prof"].W_MHD.item()
Wb = case_b["ev_prof"].W_MHD.item()
print(
    f"compare W_MHD: |1-W(a)/W(b)|= {np.abs(1-Wa/Wb):.4e}, W(a)={Wa:.7e},W(b)={Wb:.7e},"
)

In [ ]:
# gvec.comp.table_of_quantities(markdown=True)

In [ ]:
# === Profiles === #
fig, axs = plt.subplots(2, 3, figsize=(15, 6), tight_layout=True, sharex=True)
ev_a = case_a["ev_prof"]
ev_b = case_b["ev_prof"]
for ax, var in zip(axs.flatten(), ["Phi", "iota", "p", "I_tor", "I_pol", "F_r_avg"]):
    ax.plot(ev_a.rho, ev_a[var], "k-", label=f"${ev_a[var].attrs['symbol']}$ ..a")
    ax.plot(ev_b.rho, ev_b[var], "r--", label=f"${ev_b[var].attrs['symbol']}$ ..b")

    maxreldiff = np.amax(np.abs(ev_a[var].data - ev_b[var].data)) / np.amax(
        np.abs(ev_a[var].data)
    )
    ax.set(
        title=ev_a[var].attrs["long_name"]
        + " ; "
        + r"$\frac{\max|a-b|}{\max|a|}=$"
        + f"{maxreldiff: .1e}",
        ylabel=f"${ev_a[var].attrs['symbol']}$",
    )

for ax in axs[-1, :]:
    ax.set_xlabel(f"${ev_a.rho.attrs['symbol']}$")

for ax in axs.flat:
    ax.legend()

plt.suptitle(
    f"Comparison a(black)= {case_a['title']}  vs. b(red)= {case_b['title']}, example={example}",
    figure=fig,
)

In [ ]:
# === Cross-sections === #
fig, axs = plt.subplots(
    2, 3, figsize=(15, 10), tight_layout=True, sharex=True, sharey=True
)
ev = case_a["ev_visu"]
cuts = np.linspace(0, 2 * np.pi / ev.N_FP.item(), 6, endpoint=False)
for ax, zeta in zip(axs.flat, cuts):
    for evi, ab, sty, col in zip(
        [case_a["ev_visu"], case_b["ev_visu"]],
        ["a", "b"],
        ["solid", "dashed"],
        ["black", "red"],
    ):
        ds = evi.sel(zeta=zeta, method="nearest")
        rho2d = ds.rho + ds.X1 * 0
        thet2d = ds.theta + ds.LA  # thetastar!
        ax.contour(
            ds.X1.data,
            ds.X2.data,
            rho2d,
            np.linspace(1e-4, 1 - 1e-4, 11),
            linestyles=sty,
            colors=col,
        )
        ax.contour(
            ds.X1.data,
            ds.X2.data,
            thet2d.T,
            np.linspace(-2 * np.pi, 2 * np.pi, 21),
            linestyles=sty,
            colors=col,
        )

        # xi = rho2d*np.cos(thet2d.T)
        # eta= rho2d*np.sin(thet2d.T)
        # ax.contour(ds.X1.data,ds.X2.data,xi,np.linspace(-1,1,11), linestyles=sty,colors=col)
        # ax.contour(ds.X1.data,ds.X2.data,eta,np.linspace(-1,1,11), linestyles=sty,colors=col)
        # ax.plot(ds.isel(rad=nrho-1).X1.T, ds.isel(rad=nrho-1).X2.T, linestyle=sty,color=col)

        # r = np.linspace(0, ds.rad.size - 1, 5, dtype=int)
        # ax.plot(ds.isel(rad=r).X1.T, ds.isel(rad=r).X2.T, sty)
        # t = np.linspace(0, ds.pol.size - 1, 8, dtype=int, endpoint=False)
        # ax.plot(ds.isel(pol=t).X1, ds.isel(pol=t).X2, sty)
    ax.set(
        aspect="equal",
        title=f"$\\zeta/(2\\pi) = {zeta/(2*np.pi):.2f}$",
    )

for ax in axs[-1, :]:
    ax.set_xlabel(f"${ev.X1.attrs['symbol']}$")
for ax in axs[:, 0]:
    ax.set_ylabel(f"${ev.X2.attrs['symbol']}$")

plt.suptitle(
    f"Comparison a(black)= {case_a['title']}  vs. b(red)= {case_b['title']}, example={example}",
    figure=fig,
)